In [ ]:
import pandas as pd
from datasets import Dataset
from dawid_skene_model import list2array, DawidSkeneModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from langchain.schema import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm
import numpy as np
from collections import Counter
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from huggingface_hub import login
from langchain.chat_models import ChatOpenAI
import os

In [ ]:
# Configura la chiave API di OpenAI
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Imposta la variabile d'ambiente
os.environ["HUGGINGFACE_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")
# Esegui l'autenticazione
login(token=os.environ["HUGGINGFACE_TOKEN"])

# Autenticazione google
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [ ]:
# FUNZIONI AUSILIARIE
# Converti il DataFrame in una lista nidificata (dataset_list)
def dataframe_to_dataset_list(df, model_columns):
    dataset_list = []
    for _, row in df.iterrows():
        task = []
        for model in model_columns:
            response = row[model]  # Prendi la risposta del modello
            task.append([0] if response == "No" else [1])  # Converti in formato numerico
        dataset_list.append(task)
    return dataset_list

In [ ]:
# Load the dataset
eval_dataset = Dataset.load_from_disk("eval_dataset")

# Convert to DataFrame
eval_df_syntethic = eval_dataset.to_pandas()
eval_df_syntethic = eval_df_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]

# Load the dataset
eval_dataset_adjacent_chunks = Dataset.load_from_disk("eval_dataset_adjacent_chunks")

# Convert to DataFrame
eval_df_adjacent_chunks_syntethic = eval_dataset_adjacent_chunks.to_pandas()
eval_df_adjacent_chunks_syntethic = eval_df_adjacent_chunks_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]

# Load the dataset
eval_dataset_random_chunks = Dataset.load_from_disk("eval_dataset_random_chunks")

# Convert to DataFrame
eval_df_random_chunks_syntethic = eval_dataset_random_chunks.to_pandas()
eval_df_random_chunks_syntethic = eval_df_random_chunks_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]

eval_df_syntethic = eval_df_syntethic[eval_df_syntethic['chunk_num'].notna()]
eval_df_syntethic = eval_df_syntethic.reset_index(drop=True)
eval_df_syntethic['chunk_num'] = eval_df_syntethic['chunk_num'].apply(lambda x: [int(x)])

# Concatenate the DataFrames
df = pd.concat([
    eval_df_random_chunks_syntethic, 
    eval_df_syntethic, 
    eval_df_adjacent_chunks_syntethic
], ignore_index=True)

df = df.drop(columns = ["source_doc", "context", "chunk_num"])

display(df)

In [ ]:
# ** Configurazione modelli **
MODEL_NAMES = {
    "BERTino": "dbmdz/bert-base-italian-xxl-cased", #111M params
    "UmBERTo": "Musixmatch/umberto-commoncrawl-cased-v1", #110M params
    "GePpeTto": "LorenzoDeMattei/GePpeTto" #117M params
}
    
class HuggingFaceModel:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        
        # **Aggiungi un token di padding se non presente**
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
            self.model.resize_token_embeddings(len(self.tokenizer))  # Aggiorna la dimensione dei token nel modello

    def generate(self, question, answer):
        """Esegue la valutazione del modello sulla coppia domanda-risposta."""
        inputs = self.tokenizer(
            f"Domanda: {question} Risposta: {answer}", 
            return_tensors="pt", 
            truncation=True,  
            max_length=512,  
            padding="max_length"
        )
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        return "Sì" if predicted_class == 1 else "No"

# ** Inizializzazione dei modelli dell'Ensemble **
ensemble_models = {name: HuggingFaceModel(model) for name, model in MODEL_NAMES.items()}

# ** Inizializza il modello Gemini **
model_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0
)

# Inizializza il modello di ChatGPT-4o
chatgpt_model = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [ ]:
def evaluate_pair(model, row):
    messages = [
        SystemMessage(content="Sei un critico che valuta coppie di domande e risposte per una FAQ di un software gestionale. \
            Le coppie di domande e risposte devono soddisfare i seguenti criteri per essere considerate utili per valutare un chatbot destinato al supporto clienti di un software gestionale: \
            1. **Rilevanza**: Devono affrontare temi rilevanti per gli utenti di un software gestionale. \
            2. **Logicità e utilità**: Devono essere logiche e utili per fornire informazioni chiare e pratiche agli utenti. \
            Valuta la seguente coppia e decidi se è utile per testare un chatbot per il supporto clienti. \
            Rispondi esclusivamente con 'Sì' o 'No'."),
        
        HumanMessage(content=f"**Domanda:** {row['question']}\n**Risposta:** {row['answer']}\n\nRispondi esclusivamente con 'Sì' o 'No'.")
    ]

    if isinstance(model, ChatOpenAI) or isinstance(model, ChatGoogleGenerativeAI):
        response = model.invoke(messages)  # Usa invoke() per evitare errori di compatibilità
        answer = response.content.strip()
        return "Sì" if "Sì" in answer else "No"
    else:
        return model.generate(row['question'], row['answer'])


In [ ]:
tqdm.pandas()

# ** Valutazione con Gemini **
df["Gemini"] = df.progress_apply(lambda row: evaluate_pair(model_gemini, row), axis=1)

# ** Valutazione con l'ensemble di modelli piccoli **
for model_name, model in ensemble_models.items():
    df[model_name] = df.progress_apply(lambda row: evaluate_pair(model, row), axis=1)

# **Funzione per Majority Voting**
def majority_voting(row):
    votes = [row[model] for model in MODEL_NAMES.keys()]
    return Counter(votes).most_common(1)[0][0]  # Opzione più votata

# **Calcoliamo Majority Voting**
df["Majority Voting"] = df.apply(majority_voting, axis=1)

# **Confrontiamo con Gemini**
df["Agreement Majority-Gemini"] = df["Majority Voting"] == df["Gemini"]

# **Calcoliamo le percentuali di accordo**
majority_vs_gemini = df["Agreement Majority-Gemini"].mean() * 100

# ** Encoding delle risposte per l'analisi di consenso **
def encode_answers(df):
    return df.replace({"Sì": 1, "No": 0})

#encoded_df = encode_answers(df.iloc[:, 1:-1])  # Escludiamo la colonna delle domande
encoded_df = encode_answers(df.iloc[:, 1:-1]).apply(pd.to_numeric, errors="coerce")
display(encoded_df)

In [ ]:
# ** Valutazione con GPT-4o **
df["ChatGPT-4o"] = df.progress_apply(lambda row: evaluate_pair(chatgpt_model, row), axis=1)

# ** Confronto tra GPT e Gemini **
df["Agreement Gemini-GPT4o"] = df["Gemini"] == df["ChatGPT-4o"]
gpt4o_vs_gemini = df["Agreement Gemini-GPT4o"].mean() * 100

In [ ]:
# Definisci le colonne dei modelli
model_columns = ["BERTino", "UmBERTo", "GePpeTto"]

# 3Converti il DataFrame in dataset_list
dataset_list = dataframe_to_dataset_list(df, model_columns)

# Converti in tensore NumPy
class_num = 2  # Solo due classi: Sì (1) e No (0)
dataset_tensor = list2array(class_num, dataset_list)

# Inizializza e lancia il modello di Dawid & Skene
model = DawidSkeneModel(class_num=2, max_iter=40, tolerance=1e-5)
marginal_predict, error_rates, worker_reliability, predict_label = model.run(dataset_tensor)

# Converti le predizioni finali in "Sì" o "No"
final_answers = ["Sì" if p[1] > 0.5 else "No" for p in predict_label]

# Aggiungi i risultati al DataFrame
df["Dawid & Skene Multi-Class"] = final_answers

# Mostra il confronto con Majority Voting e Gemini
df["Agreement D&S Multi-Class-Gemini"] = df["Dawid & Skene Multi-Class"] == df["Gemini"]

# Calcola le percentuali di accordo
ds_multi_vs_gemini = df["Agreement D&S Multi-Class-Gemini"].mean() * 100

summary_df = pd.DataFrame({
    "Metodo": ["Majority Voting", "Dawid & Skene Multi-Class", "ChatGPT-4o"],
    "Concordanza con Gemini (%)": [majority_vs_gemini, ds_multi_vs_gemini, gpt4o_vs_gemini]
})

display(summary_df)
import seaborn as sns
import matplotlib.pyplot as plt

# Convertiamo le risposte in valori numerici per la heatmap
heatmap_df = df[[
    "Majority Voting", 
    "Dawid & Skene Multi-Class",
    "ChatGPT-4o", 
    "Gemini"
]].map(lambda x: 1 if x == "Sì" else 0)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_df, annot=True, fmt="d", cmap="coolwarm", cbar=True)
plt.title("Confronto tra Modelli e Gemini")
plt.xlabel("Metodo di Ensemble o Modello")
plt.ylabel("Domande")
plt.show()


# Other ensembles experiments

In [ ]:
# Aggiungiamo LLaMA 3.2 al dizionario dei modelli
MODEL_NAMES["LLaMA3.2"] = "meta-llama/Llama-3.2-1B" #1B

# Inizializzazione del modello LLaMA 3.2
ensemble_models["LLaMA3.2"] = HuggingFaceModel(MODEL_NAMES["LLaMA3.2"])

df["LLaMA3.2"] = df.progress_apply(lambda row: evaluate_pair(ensemble_models["LLaMA3.2"], row), axis=1)

In [ ]:
MODEL_NAMES["GPT2-smallIta"] = "GroNLP/gpt2-small-italian" #121M

ensemble_models["GPT2-smallIta"] = HuggingFaceModel(MODEL_NAMES["GPT2-smallIta"])

df["GPT2-smallIta"] = df.progress_apply(lambda row: evaluate_pair(ensemble_models["GPT2-smallIta"], row), axis=1)

In [ ]:
# Definiamo i modelli nell’ensemble
model_columns_enhanced = ['BERTino', 'UmBERTo', 'GePpeTto', 'GPT2-smallIta', 'LLaMA3.2']
def majority_voting_enhanced(row):
    models_to_consider = model_columns_enhanced  # Updated models list
    votes = [row[model] for model in models_to_consider]
    return Counter(votes).most_common(1)[0][0]  # Returns the most common choice

# **Calcoliamo Majority Voting**
df["Enhanced Majority Voting"] = df.apply(majority_voting_enhanced, axis=1)

# **Confrontiamo con Gemini**
df["Agreement Enhanced Majority-Gemini"] = df["Enhanced Majority Voting"] == df["Gemini"]

# **Calcoliamo le percentuali di accordo**
ensemble_enhanced_mv_vs_gemini = df["Agreement Enhanced Majority-Gemini"].mean() * 100

# Converti il DataFrame in lista nidificata
dataset_list_enhanced = dataframe_to_dataset_list(df, model_columns_enhanced)

# Converti in tensore NumPy per Dawid & Skene
class_num = 2  # Solo due classi: Sì (1) e No (0)
dataset_tensor_enhanced = list2array(class_num, dataset_list_enhanced)

# Inizializza e lancia il modello Dawid & Skene
ds_model_enhanced = DawidSkeneModel(class_num=2, max_iter=40, tolerance=1e-5)
marginal_predict_enhanced, error_rates_enhanced, worker_reliability_enhanced, predict_label_enhanced = ds_model_enhanced.run(dataset_tensor_enhanced)

# Converti le predizioni finali in "Sì" o "No"
final_answers_enhanced = ["Sì" if p[1] > 0.5 else "No" for p in predict_label_enhanced]

# Aggiungi i risultati al DataFrame
df["Ensemble Enhanced (Dawid & Skene)"] = final_answers_enhanced

df["Agreement Ensemble Enhanced (D&S)-Gemini"] = df["Ensemble Enhanced (Dawid & Skene)"] == df["Gemini"]
ensemble_enhanced_ds_vs_gemini = df["Agreement Ensemble Enhanced (D&S)-Gemini"].mean() * 100

summary_df = pd.DataFrame({
    "Metodo": ["Majority Voting", "Dawid & Skene Multi-Class", "Enhanced Majority Voting", "Ensemble Enhanced (Dawid & Skene)", "ChatGPT-4o"],
    "Concordanza con Gemini (%)": [majority_vs_gemini, ds_multi_vs_gemini, ensemble_enhanced_mv_vs_gemini, ensemble_enhanced_ds_vs_gemini, gpt4o_vs_gemini]
})

display(summary_df)

In [ ]:
# Convertiamo le risposte in valori numerici per la heatmap
heatmap_df = df[[
    "Majority Voting", 
    "Dawid & Skene Multi-Class",
    "Enhanced Majority Voting", 
    "Ensemble Enhanced (Dawid & Skene)",
    "ChatGPT-4o", 
    "Gemini"
]].map(lambda x: 1 if x == "Sì" else 0)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_df, annot=True, fmt="d", cmap="coolwarm", cbar=True)
plt.title("Confronto tra Modelli e Gemini")
plt.xlabel("Metodo di Ensemble o Modello")
plt.ylabel("Domande")
plt.show()

In [ ]:
# **Distribuzione delle risposte per ciascun modello**
print("\nDistribuzione delle risposte per ciascun modello:")
for model in model_columns_enhanced:
    print(f"{model}:")
    print(df[model].value_counts(normalize=True) * 100, "\n")

# **Distribuzione delle risposte per Gemini**
print("\nDistribuzione delle risposte di Gemini:")
print(df["Gemini"].value_counts(normalize=True) * 100)

# **Distribuzione delle risposte per GPT**
print("\nDistribuzione delle risposte di ChatGPT-4o:")
print(df["ChatGPT-4o"].value_counts(normalize=True) * 100)

## Variants

In [ ]:
# Definiamo i modelli nell’ensemble
model_columns_enhanced = ['UmBERTo', 'GePpeTto']
def majority_voting_enhanced(row):
    models_to_consider = model_columns_enhanced  # Updated models list
    votes = [row[model] for model in models_to_consider]
    return Counter(votes).most_common(1)[0][0]  # Returns the most common choice

# **Calcoliamo Majority Voting**
df["Enhanced3 Majority Voting"] = df.apply(majority_voting_enhanced, axis=1)

# **Confrontiamo con Gemini**
df["Agreement Enhanced3 Majority-Gemini"] = df["Enhanced3 Majority Voting"] == df["Gemini"]

# **Calcoliamo le percentuali di accordo**
ensemble_enhanced_mv_vs_gemini3 = df["Agreement Enhanced3 Majority-Gemini"].mean() * 100

# Converti il DataFrame in lista nidificata
dataset_list_enhanced = dataframe_to_dataset_list(df, model_columns_enhanced)

# Converti in tensore NumPy per Dawid & Skene
class_num = 2  # Solo due classi: Sì (1) e No (0)
dataset_tensor_enhanced = list2array(class_num, dataset_list_enhanced)

# Inizializza e lancia il modello Dawid & Skene
ds_model_enhanced = DawidSkeneModel(class_num=2, max_iter=40, tolerance=1e-5)
marginal_predict_enhanced, error_rates_enhanced, worker_reliability_enhanced, predict_label_enhanced = ds_model_enhanced.run(dataset_tensor_enhanced)

# Converti le predizioni finali in "Sì" o "No"
final_answers_enhanced = ["Sì" if p[1] > 0.5 else "No" for p in predict_label_enhanced]

# Aggiungi i risultati al DataFrame
df["Ensemble Enhanced3 (Dawid & Skene)"] = final_answers_enhanced

df["Agreement Ensemble Enhanced3 (D&S)-Gemini"] = df["Ensemble Enhanced3 (Dawid & Skene)"] == df["Gemini"]
ensemble_enhanced_ds_vs_gemini3 = df["Agreement Ensemble Enhanced3 (D&S)-Gemini"].mean() * 100

summary_df = pd.DataFrame({
    "Metodo": ["Majority Voting", "Dawid & Skene Multi-Class", "Enhanced3 Majority Voting", "Ensemble Enhanced3 (Dawid & Skene)", "ChatGPT-4o"],
    "Concordanza con Gemini (%)": [majority_vs_gemini, ds_multi_vs_gemini, ensemble_enhanced_mv_vs_gemini3, ensemble_enhanced_ds_vs_gemini3, gpt4o_vs_gemini]
})

display(summary_df)

In [ ]:
# Convertiamo le risposte in valori numerici per la heatmap
heatmap_df = df[[
    "Majority Voting", 
    "Dawid & Skene Multi-Class",
    "Enhanced3 Majority Voting", 
    "Ensemble Enhanced3 (Dawid & Skene)",
    "ChatGPT-4o", 
    "Gemini"
]].map(lambda x: 1 if x == "Sì" else 0)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_df, annot=True, fmt="d", cmap="coolwarm", cbar=True)
plt.title("Confronto tra Modelli e Gemini")
plt.xlabel("Metodo di Ensemble o Modello")
plt.ylabel("Domande")
plt.show()